## Setup

In [1]:
#!pip install -q --no-warn-conflicts malariagen_data -U

This file accesses MalariaGen Pf7 data QC pass data at the variant level to calculate site missingness across all genes in a specified gene set. It accesses Pf7 data stored via the cloud, which requires registration on the malariagen website and subsequent gcloud logins during each session. 

In [ ]:
!gcloud auth application-default login

In [ ]:
import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
import allel
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import malariagen_data
import collections
import pandas as pd
import xarray as xr
import os

pf7 = malariagen_data.Pf7()
pf7_metadata = pf7.sample_metadata()
#pf7_metadata = pd.read_csv('Pf7_samples.txt', sep = '\t')
variant_dataset = pf7.variant_calls()
genome_features0 = pf7.genome_features()
# EXTENDED VARIANT DATA -- variant_ANN_Annotation_Impact
extended_variant_dataset = pf7.variant_calls(extended=True)
path_to_gene_set = '' # insert path to desired gene set 
os.chdir(path_to_gene_set)
fn_out = 'variant_pass_rates.csv' # insert output name for tmp file with pass rates for merging with site counts

In [ ]:
def pass_rate(stage):
  # read in stage-specific gene set
  fname = stage + '.txt'
  gene_set = np.loadtxt(fname, dtype = str)
  print(stage, len(gene_set))
  # mark which genes are in gene set from BOOL
  in_set = [(i in gene_set) for i in genome_features0['ID']]
  # subset gene set features from list of all 3D7 genome features
  genome_features = genome_features0[in_set]
  contigs = genome_features['contig']
  start = genome_features['start']
  end = genome_features['end']

  # APPROACH: PREPARE ALL MASKS AND COMBINE THEM TOGETHER AT END
  # load in filter pass flags
  fpass = variant_dataset['variant_filter_pass'].data
  # variant pass dataset --> keep SNPs only (same for all)
  # first: snp mask (filter array for SNPs only)
  snp_mask = variant_dataset['variant_is_snp'].data

  # coding mask: ensure variant falls in coding region
  coding_mask = variant_dataset['variant_CDS'].data

  pass_rates = []
  # iterate through all gene coordinates pulled from reference genome
  for i, j, k in zip(contigs, start, end):
    # mask those on same contig
    contig_mask = variant_dataset['variant_chrom'].data == i
    # mask for position
    pos_mask = da.isin(variant_dataset['variant_position'].data, np.arange(j, k))
    # combine masks
    gene_mask = da.logical_and(contig_mask, pos_mask)
    gene_mask_coding = da.logical_and(gene_mask, coding_mask)
    mask_final = da.logical_and(snp_mask, gene_mask_coding)
    fpass_gene = fpass[mask_final.compute()]
    pass_rate0 = da.sum(fpass_gene).compute()/fpass_gene.size
    pass_rates.append(pass_rate0)
    #print(pass_rate0)
  return(pass_rates)

In [ ]:
# calculate pass rates for desired gene set 
rates_all = pass_rate("min_expression_2.5")

# save all to csv 
rdf = pd.DataFrame(rates_all).T
rdf.columns = ['value'] # variant pass rate coded as value 
rdf.head()
rdf.to_csv(fn_out)

In [ ]:
# runIF new session for analysis
import pandas as pd
rdfm = pd.read_csv(fn_out, index_col=0)

In [ ]:
fname = 'min_expression_2.5.txt'
genes = np.loadtxt(fname, dtype = str)

array(['PF3D7_0102600', 'PF3D7_0102700', 'PF3D7_0102800', ...,
       'PF3D7_1475000', 'PF3D7_API01900', 'PF3D7_API03500'], dtype='<U14')

In [ ]:
# combine with DF containing site counts 
props = pd.read_csv('PlasmoDB-61_Pfalciparum3D7_AnnotatedCDSs_SynAndNonsynSiteCount.txt', sep = '\t')
fname = 'min_expression_2.5.txt'
genes = np.loadtxt(fname, dtype = str)
rdfm['ID'] = genes
props = props.merge(rdfm, left_on='GENE', right_on='ID')
props['coding_length_adj'] = props['TOTAL_CODING_LENGTH'] * props['value']
# save merged file 
#props.to_csv('props_adj_breadth.txt', index=False, sep='\t')

In [ ]:
props = pd.read_csv("props_adj_breadth.txt", sep='\t') # example output--with preliminary breadth labels in "variable" column
props.head()

,GENE,TRANS,NAME,NS,SYN,FFD,TOTAL_CODING_LENGTH,PROP_NS,PROP_SYN,PROP_FFD,STOP_CODONS,FULL_LENGTH,COORD,variable,value,ID,coding_length_adj
0,PF3D7_1312500,PF3D7_1312500.1,"conserved Plasmodium protein, unknown function",146.333333,51.666667,35,198,0.739057,0.260943,0.176768,1,198,location=Pf3D7_13_v3:526863-527060(-),1,0.920000,PF3D7_1312500,182.160000
1,PF3D7_1312450,PF3D7_1312450.1,"apical ring associated protein 1, putative",144.000000,48.000000,33,192,0.750000,0.250000,0.171875,1,192,location=Pf3D7_13_v3:525028-525219(-),1,0.868421,PF3D7_1312450,166.736842
2,PF3D7_0401800,PF3D7_0401800.1,"Plasmodium exported protein (PHISTb), unknown ...",1269.500000,413.500000,288,1683,0.754308,0.245692,0.171123,1,1862,location=Pf3D7_04_v3:103742-105603(-),4,0.764706,PF3D7_0401800,1287.000000
3,PF3D7_0320900,PF3D7_0320900.1,histone H2A.Z,359.000000,118.000000,81,477,0.752621,0.247379,0.169811,1,1083,location=Pf3D7_03_v3:875213-876295(+),6,0.806818,PF3D7_0320900,384.852273
4,PF3D7_0516200,PF3D7_0516200.1,40S ribosomal protein S11,343.333333,112.666667,77,456,0.752924,0.247076,0.168860,1,806,location=Pf3D7_05_v3:675809-676614(-),4,0.747253,PF3D7_0516200,340.747253
